In [23]:
## Connect to a database
import pandas as pd
# Import sqlalchemy's create_engine() function
from sqlalchemy import create_engine

# Create the database engine
engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{databasename}')

# View the tables in the database
print(engine.table_names())
print(len(engine.table_names()))

C:\Users\Nana Tieku\AppData\Local\Temp\ipykernel_30068\2472826684.py:10: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())


['relation', 'spatial_ref_sys', 'schema_info', 'users', 'ways', 'way_nodes', 'relations', 'relation_members', 'nodes', 'my_table3']
10


C:\Users\Nana Tieku\AppData\Local\Temp\ipykernel_30068\2472826684.py:11: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(len(engine.table_names()))


In [24]:
# Create a SQL query to load the entire weather table
query = """
SELECT * 
  FROM ways;
"""

# Load weather with the SQL query
df = pd.read_sql(query, engine)

# View the first few rows of data
df.head()

,id,version,user_id,tstamp,changeset_id,tags,nodes
0,4345043,18,0,2021-05-03 14:13:03,0,"{'name': 'Kraftwerk Zolling', 'power': 'plant'...","[26464131, 1083557623, 1248745246, 1248745140,..."
1,4670340,20,0,2021-05-03 14:13:03,0,"{'name': 'Kraftwerk Bergkamen', 'power': 'plan...","[29668837, 1100238681, 8394499281, 8394499280,..."
2,4674139,46,0,2021-09-26 10:06:39,0,"{'lit': 'yes', 'ref': 'L 149', 'name': 'Molker...","[1204055483, 9048379288, 2682483361, 229510648..."
3,4677631,35,0,2021-11-15 13:51:51,0,"{'name': 'Heizkraftwerk Moabit', 'power': 'pla...","[29707202, 29707203, 4204602410, 1391402143, 4..."
4,4763495,34,0,2022-02-08 09:40:44,0,"{'name': 'Heizkraftwerk Charlottenburg', 'powe...","[2793532638, 30424002, 3240491933, 280934822, ..."


In [25]:
#Uploading a dataframe to s3 using boto3
import boto3
from credentials import access_key, secret_key
import io
import os

s3_client = boto3.client('s3',
            aws_access_key_id=access_key, 
            aws_secret_access_key= secret_key,
                       region_name="us-east-1")

with io.StringIO() as csv_buffer:
    df.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket='asw-demos-hope', Key="test.csv", Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful Upload. Status - {status}")
    else:
        print(f"Unsuccessful Upload. Status - {status}")

Successful Upload. Status - 200
